### **데이터 시각화**

공통 (환경설정 & 데이터 로드)

In [1]:
# 공통 (환경설정 & 데이터 로드) 및 문장 길이 분포 분석/시각화 통합

# ===================== 공통: 환경 세팅 및 데이터 로드 =====================
import os, re, math, random, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from transformers import AutoTokenizer # 토크나이저 임포트
import seaborn as sns # seaborn 임포트
import matplotlib.font_manager as fm # 폰트 매니저 임포트
from IPython.display import Image, display # 이미지 표시 임포트

# 경로 설정
DATA_DIR = r"/content/" # DATA_DIR을 /content/로 수정
train_path = os.path.join(DATA_DIR, "train.csv")
test_path = os.path.join(DATA_DIR, "test.csv")
lex_path   = os.path.join(DATA_DIR, "general_data.txt")

# 결과 폴더
OUT_DIR = os.path.join(DATA_DIR, "outputs")
FIG_DIR = os.path.join(OUT_DIR, "figures")
TAB_DIR = os.path.join(OUT_DIR, "tables")
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(TAB_DIR, exist_ok=True)

# 시각화 공통 함수
def savefig_tight(path):
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    print(f"📁 저장됨: {path}")
    plt.close()

# 폰트 설정 (이전 오류 해결 내용 반영)
# Nanum 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
if os.path.exists(fontpath):
    fm.fontManager.addfont(fontpath) # 폰트를 강제로 추가합니다.
    plt.rc('font', family='NanumBarunGothic') # 재확인 차원에서 다시 설정
    plt.rcParams['axes.unicode_minus'] = False
    print("✅ Matplotlib 폰트 설정 완료: NanumBarunGothic")
else:
    print(f"❌ 폰트 파일이 다음 경로에 존재하지 않습니다: {fontpath}\n한글 폰트가 깨져 보일 수 있습니다.")

# 데이터 로드
df_train = pd.read_csv(train_path)
df_train = df_train.dropna(subset=["conversation"])
df_train = df_train.drop_duplicates(subset=["class","conversation"]).reset_index(drop=True)

# general_data.txt 로드 및 통합
df_general = pd.read_csv(lex_path)
df_general = df_general.dropna(subset=["conversation"])
df_general = df_general.drop_duplicates(subset=["class","conversation"]).reset_index(drop=True)

df = pd.concat([df_train, df_general], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True) # 데이터 섞기

print("\n--- 통합 데이터프레임 미리보기 ---")
print(df.head())
print("\n--- 클래스 분포 ---", df["class"].value_counts())

# general_data.txt 파일 내용 미리보기
print(f"\n--- 'general_data.txt' 파일 내용 미리보기 ({lex_path}) ---")
if os.path.exists(lex_path):
    with open(lex_path, 'r', encoding='utf-8') as f:
        content = f.read()
    print(content[:500]) # 파일의 처음 500자만 출력하여 내용을 확인합니다.
    print("\n(파일 내용이 길어 일부만 출력했습니다.)")
else:
    print(f"❌ 'general_data.txt' 파일이 다음 경로에 존재하지 않습니다: {lex_path}")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (12.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 125082 files and direc

** 임베딩 시각화 (t-SNE / UMAP)**

In [5]:
# ===================== 4. 임베딩 시각화 =====================
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import umap, plotly.express as px

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
emb = model.encode(df["conversation"].tolist(), show_progress_bar=True, normalize_embeddings=True)

# UMAP
umap_2d = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, metric="cosine", random_state=42).fit_transform(emb)
umap_df = pd.DataFrame({"x":umap_2d[:,0],"y":umap_2d[:,1],"class":df["class"]})
fig = px.scatter(umap_df, x="x", y="y", color="class", title="UMAP 2D by Class", opacity=0.8)
fig.write_html(os.path.join(FIG_DIR,"umap_2d.html"))

# t-SNE
tsne_2d = TSNE(n_components=2, perplexity=30, random_state=42, init="pca", learning_rate="auto").fit_transform(emb)
tsne_df = pd.DataFrame({"x":tsne_2d[:,0],"y":tsne_2d[:,1],"class":df["class"]})
fig2 = px.scatter(tsne_df, x="x", y="y", color="class", title="t-SNE 2D by Class", opacity=0.8)
fig2.write_html(os.path.join(FIG_DIR,"tsne_2d.html"))

print("✅ UMAP & t-SNE 시각화 완료! HTML 파일은 figures 폴더에서 확인 가능.")

Batches:   0%|          | 0/146 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



✅ UMAP & t-SNE 시각화 완료! HTML 파일은 figures 폴더에서 확인 가능.


In [6]:
from IPython.display import HTML

umap_html_path = os.path.join(FIG_DIR, "umap_2d.html")

if os.path.exists(umap_html_path):
    with open(umap_html_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    display(HTML(html_content))
    print(f"✅ '{umap_html_path}' 파일이 성공적으로 표시되었습니다.")
else:
    print(f"❌ 오류: '{umap_html_path}' 파일을 찾을 수 없습니다. UMAP 시각화 파일을 먼저 생성해주세요.")

Output hidden; open in https://colab.research.google.com to view.

## UMAP과 t-SNE의 데이터 분석 결과 차이점

데이터 분석 결과, UMAP과 t-SNE 시각화에서 관찰된 주요 차이점은 다음과 같습니다:

*   **전반적인 구조 표현**: UMAP은 전역적인 구조(클러스터 간의 큰 그림 관계)를 더 잘 보존하는 경향이 있는 반면, t-SNE는 각 클러스터 내부의 **지역적인 구조**를 더 상세하고 밀집된 형태로 보여주는 데 탁월합니다.

*   **클러스터 간 거리**: UMAP 플롯에서는 클러스터 간의 상대적인 거리가 t-SNE보다 더 의미 있게 해석될 수 있습니다. t-SNE는 지역 구조 보존에 집중하기 때문에, 서로 멀리 떨어진 클러스터 간의 거리는 왜곡될 수 있습니다.

*   **중첩 양상**: '갈취', '협박', '직장 내 괴롭힘', '기타 괴롭힘'과 같은 괴롭힘 관련 대화 유형의 경우, 두 시각화 모두 상당한 중첩을 보였지만, t-SNE는 때때로 UMAP보다 더 복잡하고 밀집된 클러스터 구조를 나타내어 미묘한 지역적 차이를 더 잘 드러낼 수 있습니다.

*   **일반 대화의 분리**: 두 기법 모두 '일반' 대화는 다른 괴롭힘 대화와 비교적 뚜렷하게 분리된 클러스터를 형성하는 것을 일관되게 보여주었습니다.

결론적으로, **'일반' 대화와 괴롭힘 대화 간의 전반적인 구분을 확인하고 싶다면 UMAP**이 더 직관적일 수 있습니다. 반면, 여러 괴롭힘 대화 유형들이 왜 서로 중첩되는지, 그 안에서 어떤 지역적 유사성이 있는지를 더 자세히 탐색하고 싶다면 t-SNE가 유용할 것입니다. 이 데이터에서는 특히 '일반' 대화가 잘 분리되어 보이는 것이 두 시각화 모두에서 일관되게 나타난 중요한 특징입니다.

In [7]:
tsne_html_path = os.path.join(FIG_DIR, "tsne_2d.html")

if os.path.exists(tsne_html_path):
    with open(tsne_html_path, 'r', encoding='utf-8') as f:
        html_content_tsne = f.read()
    display(HTML(html_content_tsne))
    print(f"✅ '{tsne_html_path}' 파일이 성공적으로 표시되었습니다.")
else:
    print(f"❌ 오류: '{tsne_html_path}' 파일을 찾을 수 없습니다. t-SNE 시각화 파일을 먼저 생성해주세요.")

Output hidden; open in https://colab.research.google.com to view.

## UMAP 및 t-SNE 임베딩 시각화 분석

표시된 UMAP 및 t-SNE 시각화 결과를 바탕으로 대화 분류를 위한 임베딩 공간의 질적 분석 수행.

### **UMAP 2D 클래스별 분석**

*   **분포 및 클러스터링**: UMAP 플롯은 일반적으로 몇 개의 뚜렷한 클러스터를 보여주며, 이는 `SentenceTransformer` 모델이 유사한 대화를 함께 그룹화할 수 있음을 시사합니다. 각기 다른 색상(클래스)은 자신만의 영역을 형성하는 경향이 있습니다.
*   **분리도**: 일부 클래스, 특히 '일반' 대화는 비교적 밀집되고 잘 분리된 클러스터를 형성하는 것으로 나타나, 임베딩 공간에서 다른 유형의 대화와 확연히 다름을 나타냅니다. '갈취 대화', '협박 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화'와 같은 다른 클래스들은 더 많은 중첩을 보입니다. 이러한 중첩은 이들 범주가 유사한 언어적 특징이나 주제적 요소를 공유하여 임베딩 유사성만으로는 구별하기 어렵게 만들 수 있음을 시사합니다.
*   **응집성**: 각 클러스터 내에서 동일 클래스의 대화를 나타내는 점들은 일반적으로 응집성 있게 나타나며, 이는 서로 가깝게 위치함을 의미합니다. 그러나 일부 클래스는 다른 클래스보다 더 넓게 퍼져 있어 내부적인 변동을 나타냅니다.
*   **이상치**: 주된 클래스 클러스터에서 멀리 떨어진 몇 개의 흩어진 점 또는 작은 점 그룹이 있습니다. 이는 할당된 클래스에 대해 비정형적인 대화이거나 잘못 분류되었을 수 있습니다.

### **t-SNE 2D 클래스별 분석**

*   **분포 및 클러스터링**: t-SNE는 종종 지역적 구조를 드러내는 데 탁월하며, 여기서는 UMAP보다 더 복잡하고 때로는 더 밀집된 클러스터를 보여줍니다. UMAP과 마찬가지로 다양한 클래스에 대해 뚜렷한 클러스터가 보입니다.
*   **분리도**: '일반' 대화는 다시 한번 비교적 뚜렷한 클러스터를 형성합니다. 그러나 '갈취 대화', '협박 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화' 간의 경계는 여전히 상당한 혼합을 보입니다. 이는 이러한 유형의 대화가 분류상의 어려움을 나타내며, 임베딩된 표현이 명확하게 분리되지 않음을 확인시켜 줍니다.
*   **응집성**: 클래스는 일반적으로 지역 클러스터 내에서 좋은 내부 응집성을 보입니다. t-SNE의 지역 이웃을 보존하는 강점은 이러한 작고 밀집된 그룹을 더욱 명확하게 만듭니다.
*   **이상치**: UMAP과 유사하게 일부 이상치가 존재하며, 이는 모호한 예시이거나 해당 클래스의 주요 부분에서 벗어나게 하는 고유한 특성을 가진 데이터 포인트일 수 있습니다.

### **대화 분류를 위한 전반적인 평가**

*   **유용성**: 두 시각화 모두 문장 임베딩이 대화 유형 간의 의미론적 차이를 포착할 수 있음을 확인시켜 줍니다. '일반' 대화는 대부분 구별 가능하며, 이는 이진 분류(예: 일반 vs. 괴롭힘)에 긍정적인 신호입니다.
*   **과제**: 다양한 유형의 괴롭힘/희롱 대화('갈취', '협박', '직장 내 괴롭힘', '기타 괴롭힘') 간의 상당한 중첩은 단순한 유사성 기반 접근 방식이 이러한 세분화된 범주를 구별하는 데 어려움을 겪을 수 있음을 나타냅니다. 이는 더 정교한 분류 모델이 필요하거나, 이러한 범주가 대화 텍스트 자체를 넘어 추가적인 맥락 또는 기능을 필요로 할 수 있음을 시사합니다.

## UMAP과 t-SNE의 주요 차이점

UMAP(Uniform Manifold Approximation and Projection)과 t-SNE(t-Distributed Stochastic Neighbor Embedding)는 모두 고차원 데이터를 2차원 또는 3차원과 같은 저차원 공간으로 시각화하여 데이터의 구조를 파악하는 데 유용한 비선형 차원 축소 기법입니다. 하지만 두 기법은 다음과 같은 주요 차이점을 가집니다.

1.  **알고리즘 기반 및 이론적 배경:**
    *   **t-SNE**: 주로 **지역적 구조(local structure)** 보존에 중점을 둡니다. 데이터 포인트 간의 유사성을 조건부 확률로 모델링하고, 저차원 공간에서 이 확률 분포를 최대한 유사하게 유지하도록 최적화합니다. 이는 각 데이터 포인트의 이웃을 잘 보존하는 데 효과적입니다.
    *   **UMAP**: **매니폴드 학습(manifold learning)** 및 **위상수학(topology)** 이론에 기반을 둡니다. 이는 데이터를 저차원 매니폴드에 임베딩하는 것을 목표로 하며, 지역적 구조뿐만 아니라 **전역적 구조(global structure)** 보존에도 더 강점을 가집니다. UMAP은 모든 스케일에서 데이터의 연결성을 모델링하여 데이터를 압축하고 전역적인 데이터 관계를 더 잘 반영합니다.

2.  **전역적 vs. 지역적 구조 보존:**
    *   **t-SNE**: 지역적 구조(즉, 가까운 점들은 저차원에서도 가깝게 유지)를 매우 잘 보존하지만, 클러스터 간의 상대적인 거리나 전체적인 데이터의 전역적 구조는 왜곡될 수 있습니다. 서로 멀리 떨어진 클러스터 간의 의미 있는 거리를 보장하기 어렵습니다.
    *   **UMAP**: 지역적 구조와 전역적 구조를 모두 보존하는 데 더 나은 균형을 제공합니다. 이는 더 큰 클러스터 구조와 클러스터 간의 관계를 t-SNE보다 더 정확하게 나타낼 수 있습니다.

3.  **계산 복잡도 및 속도:**
    *   **t-SNE**: 데이터 포인트 수가 증가함에 따라 계산 복잡도가 $O(N^2)$에 가까워져 대규모 데이터셋에서는 매우 느릴 수 있습니다. (N은 데이터 포인트 수)
    *   **UMAP**: 계산 복잡도가 $O(N 	ext{log}N)$ 또는 $O(N)$에 가까워 t-SNE보다 훨씬 빠릅니다. 대규모 데이터셋에 대한 확장성이 더 좋습니다.

4.  **확정성 (Determinism):**
    *   **t-SNE**: 무작위 초기화로 인해 실행할 때마다 결과가 약간씩 달라질 수 있습니다. (시드를 고정하면 재현 가능)
    *   **UMAP**: `random_state`를 설정하면 t-SNE보다 더 일관된 결과를 제공하며, 일반적으로 더 확정적인 결과를 보입니다.

5.  **파라미터 민감도:**
    *   **t-SNE**: `perplexity`와 `learning_rate` 파라미터에 매우 민감하며, 이 파라미터들을 잘 튜닝해야 좋은 시각화 결과를 얻을 수 있습니다.
    *   **UMAP**: `n_neighbors`와 `min_dist` 파라미터가 t-SNE의 파라미터보다 해석하기 쉽고 덜 민감하여, 일반적으로 더 쉽게 좋은 결과를 얻을 수 있습니다.

**요약하자면,** t-SNE는 지역적 패턴을 상세하게 탐색하는 데 뛰어나지만, UMAP은 대규모 데이터셋에서 전역적 구조를 더 효율적이고 안정적으로 시각화하는 데 더 적합합니다. 따라서 데이터의 크기와 분석 목적에 따라 적절한 기법을 선택하는 것이 중요합니다.